# Classifier NNs

## NNs summary
- **Input: an uncertainty realization vector.**
- **Output: a label vector with multiple choices.**



- NNs components
    - Dense
    - BatchNormalization
    - DropOut
    - SoftMax

- Optimizer
    - Adam

- Loss
    - BinaryCrossEntropy

- Metric
    - Accuracy [%]

In [1]:
import pickle
import pprint
from sklearn.model_selection import train_test_split
from os import path

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
test_cases = [
    'pglib_opf_case24_ieee_rts.pickle', 
    'pglib_opf_case73_ieee_rts.pickle',
    'pglib_opf_case162_ieee_dtc.pickle',
    'pglib_opf_case300_ieee.pickle',
]

In [3]:
# choose a dataset
case_idx = 3
file_dir = path.join('./datasets/', test_cases[case_idx])

In [4]:
infile = open(file_dir,'rb')
dataset = pickle.load(infile)
infile.close()

In [5]:
# train & test
x_train, x_test, y_train, y_test = train_test_split(dataset['x'],
                                                    dataset['y'],
                                                    test_size=0.2,
                                                    random_state=19)
# train & validation
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=19)

In [6]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
y_test = y_test.astype('float32')

In [7]:
print((x_train.shape, y_train.shape))
print((x_val.shape, y_val.shape))
print((x_test.shape, y_test.shape))

((32000, 300), (32000, 549))
((8000, 300), (8000, 549))
((10000, 300), (10000, 549))


In [8]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [9]:
inputs = keras.Input(shape=(x_train.shape[1], ), name='uncertainty_realization')

x = layers.Dense(x_train.shape[1], activation='relu', name='dense_1')(inputs)
x = layers.BatchNormalization(name='bn_1')(x)
x = layers.Dropout(rate=0.2, name='dropout_1')(x)

x = layers.Dense(x_train.shape[1]*2, activation='relu', name='dense_2')(x)
x = layers.BatchNormalization(name='bn_2')(x)
x = layers.Dropout(rate=0.2, name='dropout_2')(x)

x = layers.Dense(y_train.shape[1], activation='relu', name='dense_3')(x)
x = layers.BatchNormalization(name='bn_3')(x)
x = layers.Dropout(rate=0.2, name='dropout_3')(x)

outputs = layers.Dense(y_train.shape[1], activation='sigmoid', name='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="multi-label-classification")

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(),
    # List of metrics to monitor
    metrics=['accuracy'])

In [11]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [12]:
model.summary()

Model: "multi-label-classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
uncertainty_realization (Inp [(None, 300)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 300)               1200      
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 600)               180600    
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 600)               2400      
_________________________________________________________________
dropout_2 (Dropout)          (None, 600)

In [13]:
print('> Fit model on training data')
history = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=5,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
    callbacks=[tensorboard_callback])

> Fit model on training data
Train on 32000 samples, validate on 8000 samples
Epoch 1/5
32000/32000 [==============================] - 6s 200us/sample - loss: 0.0635 - accuracy: 0.9716 - val_loss: 3.5207e-04 - val_accuracy: 1.0000
Epoch 2/5
32000/32000 [==============================] - 6s 186us/sample - loss: 2.3837e-04 - accuracy: 1.0000 - val_loss: 8.0737e-05 - val_accuracy: 1.0000
Epoch 3/5
32000/32000 [==============================] - 5s 155us/sample - loss: 7.2529e-05 - accuracy: 1.0000 - val_loss: 3.1319e-05 - val_accuracy: 1.0000
Epoch 4/5
32000/32000 [==============================] - 5s 152us/sample - loss: 3.1125e-05 - accuracy: 1.0000 - val_loss: 1.4534e-05 - val_accuracy: 1.0000
Epoch 5/5
32000/32000 [==============================] - 5s 154us/sample - loss: 1.5449e-05 - accuracy: 1.0000 - val_loss: 7.4088e-06 - val_accuracy: 1.0000


In [14]:
pp = pprint.PrettyPrinter(indent=4)
print('> History dict:')
pp.pprint(history.history)

> History dict:
{   'accuracy': [0.9716356, 1.0, 1.0, 1.0, 1.0],
    'loss': [   0.0634620059877925,
                0.00023837196529348148,
                7.252880383566662e-05,
                3.1125362660532116e-05,
                1.5448769411250395e-05],
    'val_accuracy': [1.0, 1.0, 1.0, 1.0, 1.0],
    'val_loss': [   0.0003520660735666752,
                    8.073709908057935e-05,
                    3.131878317071823e-05,
                    1.4534444937453372e-05,
                    7.4087659322685795e-06]}


In [15]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 30431), started 21:15:36 ago. (Use '!kill 30431' to kill it.)

In [16]:
print('\n# Evaluate')
result = model.evaluate(x_test, y_test)
dict(zip(model.metrics_names, result))


# Evaluate
10000/10000 [==============================] - 1s 58us/sample - loss: 7.4152e-06 - accuracy: 1.0000


{'loss': 7.415209760802099e-06, 'accuracy': 1.0}

In [17]:
y_test[5]

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [18]:
y_pred = model.predict(x_test)

In [19]:
y_pred[0]

array([7.48533012e-06, 6.30202476e-06, 8.90106639e-06, 1.13880724e-05,
       7.35764615e-06, 8.08351615e-06, 9.99991059e-01, 9.99992371e-01,
       9.99990940e-01, 9.99994755e-01, 6.63743003e-06, 9.99992490e-01,
       6.91013156e-06, 8.99599581e-06, 9.99993205e-01, 5.85414318e-06,
       9.99991775e-01, 9.99993801e-01, 9.99993086e-01, 4.68119424e-06,
       9.99993086e-01, 8.87435817e-06, 7.28238547e-06, 8.22694255e-06,
       9.99988914e-01, 9.99992251e-01, 9.99992609e-01, 9.99993205e-01,
       9.20129605e-06, 9.99992371e-01, 8.45710474e-06, 9.99992251e-01,
       9.99990463e-01, 6.99261273e-06, 9.99994397e-01, 9.99994040e-01,
       9.99994040e-01, 9.99991298e-01, 9.99993801e-01, 9.99992490e-01,
       9.99992847e-01, 9.99993682e-01, 8.45908107e-06, 9.99991417e-01,
       9.99994993e-01, 9.99994040e-01, 9.99992490e-01, 9.99993443e-01,
       9.99994397e-01, 9.99992609e-01, 9.99994397e-01, 9.99992371e-01,
       9.99994278e-01, 9.99991655e-01, 6.31853845e-06, 9.99992251e-01,
      